In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
model.train(data="data.yaml", epochs=50)

In [ ]:
from ultralytics import YOLO
!yolo task=detect mode=val model='D:/project_cv/YOLOv8/runs/detect/train2/weights/best.pt' data=data.yaml

In [ ]:
!yolo task=detect mode=predict model='D:/project_cv/YOLOv8/runs/detect/train2/weights/best.pt' conf=0.15 source=D:/project_cv/datasets/data/test/images

In [1]:
!pip install requests_toolbelt

In [2]:
import os
import cv2
import time
import shutil
import math
import winsound
import requests
from PIL import Image
import io
from requests_toolbelt.multipart.encoder import MultipartEncoder
import yaml

In [3]:
TO_PREDICT_PATH         = "D:/project_cv/YOLOv8/To_Predict_Videos/"
PREDICTED_PATH          = "D:/project_cv/YOLOv8/Predicted_Videos/"
SAVE_ANNOTATED_VIDEOS   = True
MIN_SCORE               = 0.4 # Minimum object detection score

In [4]:
def time_convert(sec):
    mins = sec // 60
    sec = sec % 60
    hours = mins // 60
    mins = mins % 60
    print("Time Lapsed = {0}h:{1}m:{2}s".format(int(hours), int(mins), round(sec) ) )

In [5]:
def deleteDirContents(dir):
    # Deletes photos in path "dir"
    # # Used for deleting previous cropped photos from last run
    for f in os.listdir(dir):
        full_path = os.path.join(dir, f)
        if os.path.isdir(full_path):
            shutil.rmtree(full_path)
        else:
            os.remove(full_path)

In [6]:
start_time = time.time()
frequency = 700  
duration = 150 
for video_name in os.listdir(TO_PREDICT_PATH):
    video_path = os.path.join(TO_PREDICT_PATH, video_name)
    video_capture = cv2.VideoCapture(video_path)
    
    frame_count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    video_fps = round( video_capture.get(cv2.CAP_PROP_FPS) )
    
    success, image_b4_color = video_capture.read()
    
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    video_out = cv2.VideoWriter(PREDICTED_PATH + video_name, fourcc, video_fps, (int(image_b4_color.shape[1]),int(image_b4_color.shape[0])))
    while success:
        success, image_b4_color = video_capture.read()
        if not success:
            break
        image = cv2.cvtColor(image_b4_color, cv2.COLOR_BGR2RGB)
        
        pilImage = Image.fromarray(image)
        
        # Convert to JPEG Buffer
        buffered = io.BytesIO()
        pilImage.save(buffered, quality=100, format="JPEG")
        
        #settings = yaml.safe_load( open("config.yaml") )

        
        upload_url = "".join(["https://detect.roboflow.com/analog-u4st3/1","?api_key=XoW0tu9OH42zNFYjSkac","&confidence="+str(MIN_SCORE)])
        m = MultipartEncoder(fields={'file': ("imageToUpload", buffered.getvalue(), "image/jpeg")})
        
        response = requests.post(upload_url, 
                                 data=m, 
                                 headers={'Content-Type': m.content_type},
                                 )
        predictions = response.json()['predictions']
        # print(predictions)
        coordinates = []
        labels_found = []
        confidence_level_list = []
        for prediction in predictions:
            x1 = prediction['x'] - prediction['width']/2
            y1 = prediction['y'] - prediction['height']/2
            x2 = x1 + prediction['width']
            y2 = y1 + prediction['height']
            coordinates.append([x1,y1,x2,y2])
            label = prediction['class']
            labels_found.append(label)
        if "Center-Needle_tip" in labels_found and "needle_tip" in labels_found:

            #Center 
            center_indexes = [index for index, x in enumerate(labels_found) if x == "Center-Needle_tip"]
            center_coordinates = coordinates[center_indexes[0]]
            center_x_center = int(center_coordinates[0] + (center_coordinates[2] - center_coordinates[0]/2))
            center_y_center = int(center_coordinates[1] + (center_coordinates[3] - center_coordinates[1]/2))
            needle_tip_indexes = [index for index, x in enumerate(labels_found) if x == "needle_tip"]
            needle_tip_coordinates = coordinates[needle_tip_indexes[0]]
            center_x_needle_tip = int(needle_tip_coordinates[0] + (needle_tip_coordinates[2] - needle_tip_coordinates[0]/2))
            center_y_needle_tip = int(needle_tip_coordinates[1] + (needle_tip_coordinates[3] - needle_tip_coordinates[1]/2))
            dy = center_y_needle_tip - center_y_center
            dx = center_x_needle_tip - center_x_center
            theta = math.atan2(dy,dx)
            theta = math.degrees(theta)
            theta = round(theta)
            if theta < 0:
                theta *= -1
                theta = (180 - theta) + 180
            theta = theta - 90
            if theta < 0:
                theta *= -1
                theta = theta + 270

            if theta <= 75 or theta >= 174:
                winsound.Beep(frequency, duration)

            for label_index, label in enumerate(labels_found):
                if "Center-Needle_tip" in label:
                    labels_found[label_index] = label + " " + str(theta) + " deg"

                if "needle_tip" in label:
                    psi = 15.117*theta - 636.9
                    labels_found[label_index] = label + " " + str(psi) + " psi"
                    
            print(labels_found)        
        if SAVE_ANNOTATED_VIDEOS:
            for coordinate_index, coordinate in enumerate(coordinates):
                start_point = (int(coordinate[0]), int(coordinate[1]))
                end_point =  (int(coordinate[2]), int(coordinate[3]))
                color_1 = (255, 0, 255)
                color_2 = (255,255,255)
                thickness = 1
                cv2.rectangle(image_b4_color, start_point, end_point, color_1, thickness)
                start_point_text = (start_point[0], max(start_point[1]-5,0))
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.5
                thickness_text = 1
                cv2.putText(image_b4_color, labels_found[coordinate_index], start_point_text, font, font_scale, color_2, thickness_text)
                
                                    
        video_out.write(image_b4_color)
    video_out.release() 

['Center-Needle_tip 176 deg', 'needle_tip 2023.692 psi']
['Center-Needle_tip 176 deg', 'needle_tip 2023.692 psi']
['Center-Needle_tip 175 deg', 'needle_tip 2008.5750000000003 psi']
['Center-Needle_tip 175 deg', 'needle_tip 2008.5750000000003 psi']
['Center-Needle_tip 175 deg', 'needle_tip 2008.5750000000003 psi']
['Center-Needle_tip 175 deg', 'needle_tip 2008.5750000000003 psi']
['Center-Needle_tip 175 deg', 'needle_tip 2008.5750000000003 psi']
['Center-Needle_tip 174 deg', 'needle_tip 1993.458 psi']
['Center-Needle_tip 174 deg', 'needle_tip 1993.458 psi']
['Center-Needle_tip 173 deg', 'needle_tip 1978.341 psi']
['Center-Needle_tip 173 deg', 'needle_tip 1978.341 psi']
['Center-Needle_tip 173 deg', 'needle_tip 1978.341 psi']
['Center-Needle_tip 172 deg', 'needle_tip 1963.2240000000002 psi']
['Center-Needle_tip 172 deg', 'needle_tip 1963.2240000000002 psi']
['Center-Needle_tip 172 deg', 'needle_tip 1963.2240000000002 psi']
['Center-Needle_tip 171 deg', 'needle_tip 1948.107 psi']
['Center